# Config & Imports

In [1]:
config = {
    'data': {
        'n_datasets': 10_000, # the number of datasets
        
        'n_samples': 5_000, # the number of samples per dataset
        
        'n_features': 7, 
        # The total number of features. 
        # These comprise n_informative informative features, n_redundant redundant features, n_repeated duplicated features and 
        # n_features-n_informative-n_redundant-n_repeated useless features drawn at random.
        
        'n_informative': 6,
        # The number of informative features. Each class is composed of a number of gaussian clusters each located around the vertices 
        # of a hypercube in a subspace of dimension n_informative. For each cluster, informative features are drawn independently 
        # from N(0, 1) and then randomly linearly combined within each cluster in order to add covariance. The clusters are then 
        # placed on the vertices of the hypercube.
        
        'n_targets': 2,
        # The number of targets (or labels) of the classification problem.
    
        'n_clusters_per_class': 2,
        # The number of clusters per class.
        
        'class_sep': 4.0,
        # class_sepfloat, default=1.0
        # The factor multiplying the hypercube size. Larger values spread out the clusters/classes and make the classification task 
        # easier.
        
        'noise': 0.01,
        # flip_y (fraction of samples whose class is assigned randomly)
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': None,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },      
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.2,
                'val_size': 0.1,
                'random_state': None,
                'shuffle': True,
                'stratify': None
            }
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 32,
            'epochs': 150,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'inets': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.2,
                'val_size': 0.1,
                'random_state': None,
                'shuffle': True,
                'stratify': None
            }
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 32,
            'epochs': 1000,
            'verbose': 'auto',
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 20,
        'use_gpu': True,
        'gpu_numbers': '4',
        'RANDOM_SEED': 1,   
    }
}

In [2]:
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from tensorflow import keras
import keras.layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ReLU

from keras.regularizers import L1L2
from keras.regularizers import L2

import utilities_LR

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = config['computation']['gpu_numbers'] if config['computation']['use_gpu'] else ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' if config['computation']['use_gpu'] else ''

os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda-11.4' if config['computation']['use_gpu'] else ''#-10.1' #--xla_gpu_cuda_data_dir=/usr/local/cuda, 
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2 ,--tf_xla_enable_xla_devices' if config['computation']['use_gpu'] else ''#'--tf_xla_auto_jit=2' #, --tf_xla_enable_xla_devices

In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Num GPUs Available:  1
Num XLA-GPUs Available:  0


# Functions

In [5]:
#def load_model(datasetIndex):
 #   # with open(utilities_LR.lambda_path(config, datasetIndex), 'r') as f:
  #  #     model = keras.models.load_model(f)
   # model = keras.models.load_model(utilities_LR.lambda_path(config, datasetIndex))
    #return model

In [6]:
# def get_weightdata(datasetIndex):
  #   weights_list = load_model(datasetIndex).get_weights()
    # return np.concatenate([x.flatten() for x in weights_list])

In [7]:
def load_lambda():
    directory = utilities_LR.lambda_path_LR(config)
    
    with open(directory + '/lambda_weights_list.npy', "rb") as f:
        return np.load(f, allow_pickle=True)

In [8]:
#def get_models():
#    # return np.array([get_weights(i) for i in range(config['data']['n_datasets'])])
#    # return np.array([get_weightdata(i) for i in range(10)])
#    with open(utilities_LR.lambda_path_LR(config, 0) + '.npy', 'rb') as f:
#        weights_list = np.load(f, allow_pickle=True)
#    for i in range(1, config['data']['n_datasets']):
#        with open(utilities_LR.lambda_path(config, i) + '.npy', 'rb') as f:
#            weights = np.load(f, allow_pickle=True)
#        weights_list = np.vstack([weights_list, weights])
#    return weights_list

In [9]:
def get_coef():
    return np.load(utilities_LR.data_path_LR(config) + '/coef_list_targetForInet.npy')

In [10]:
def train_test_val_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        y, 
                                                        test_size=config['inets']['data_prep']['train_test_val_split']['test_size'] + config['inets']['data_prep']['train_test_val_split']['val_size'], 
                                                        random_state=config['inets']['data_prep']['train_test_val_split']['random_state'], 
                                                        shuffle=config['inets']['data_prep']['train_test_val_split']['shuffle'], 
                                                        stratify=config['inets']['data_prep']['train_test_val_split']['stratify'])
    X_test, X_val, y__test, y_val = train_test_split(X_test, 
                                                    y_test, 
                                                    test_size=config['inets']['data_prep']['train_test_val_split']['val_size'] / (config['inets']['data_prep']['train_test_val_split']['test_size'] + config['inets']['data_prep']['train_test_val_split']['val_size']), 
                                                    random_state=config['inets']['data_prep']['train_test_val_split']['random_state'], 
                                                    shuffle=config['inets']['data_prep']['train_test_val_split']['shuffle'], 
                                                    stratify=config['inets']['data_prep']['train_test_val_split']['stratify'])
    return X_train, X_test, X_val, y_train, y_test, y_val
    

In [11]:
def save_model(model):
    path = utilities_LR.inet_path_LR(config)
    
    Path(path).mkdir(parents=True, exist_ok=True)
    
    # weights_list = model.get_weights()
    
    # weights_linearized = np.concatenate([x.flatten() for x in weights_list])
    
    # with open(path + '/inet_weights.npy', "wb") as f:
        # np.save(f, weights_linearized, allow_pickle=True)
    model.save(path + '/modelKeras')

# Model Training

In [12]:
X = load_lambda()

In [13]:
X.shape

(10000, 7010)

In [14]:
y = get_coef()

In [15]:
y[0]

array([ 0.32982863, -0.80408867,  1.41292079, -0.75177388, -0.29918766,
       -0.51171896,  0.53138024])

In [16]:
y.shape

(10000, 7)

In [17]:
# Data Prep
X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,
                                                                      y)

# Model Def
model = Sequential()
# model.add(BatchNormalization(axis=1,
#    momentum=0.99,
#    epsilon=0.001,
#    center=True,
#    scale=True,
#    beta_initializer='zeros',
#    gamma_initializer='ones',
#    moving_mean_initializer='zeros',
#    moving_variance_initializer='ones',
#    beta_regularizer=None,
#    gamma_regularizer=None,
#    beta_constraint=None))
model.add(Dense(
    units=1000,
    kernel_regularizer=L1L2(l1=1e-5, l2=1e-4),
    bias_regularizer=L2(1e-4),
    activity_regularizer=L2(1e-5)
))
model.add(ReLU())
#model.add(Dense(6500, activation='linear'))
#model.add(Dense(6000, activation='relu'))
#model.add(Dense(5500, activation='linear'))
#model.add(Dense(3500, activation='relu'))
#model.add(Dense(2500, activation='relu'))
#model.add(Dense(1500, activation='linear'))
#model.add(Dense(500, activation='relu'))
model.add(Dense(y.shape[1], activation='relu'))


#opt = tf.keras.optimizers.Adam(
#    learning_rate=3e-4,
#    beta_1=0.999999,
#    beta_2=0.999,
#    epsilon=1e-07,
#    amsgrad=False,
#    name="Adam")

opt = tf.keras.optimizers.SGD(
    learning_rate=0.01,
    momentum=0.0,
    nesterov=False,
    name='SGD'
)

model.compile(optimizer=opt, loss='mse', metrics=['mse', keras.metrics.RootMeanSquaredError()])
model.build(input_shape=X_train.shape)
model.summary()

# Model fit
history = model.fit(x=X_train,
                    y=y_train,
                    batch_size=config['inets']['model_fit']['batch_size'],
                    epochs=config['inets']['model_fit']['epochs'],
                    verbose=config['inets']['model_fit']['verbose'],
                    callbacks=config['inets']['model_fit']['callbacks'],
                    validation_data=(X_val, y_val),
                    shuffle=config['inets']['model_fit']['shuffle'],
                    class_weight=config['inets']['model_fit']['class_weight'],
                    sample_weight=config['inets']['model_fit']['sample_weight'],
                    initial_epoch=config['inets']['model_fit']['initial_epoch'],
                    steps_per_epoch=config['inets']['model_fit']['steps_per_epoch'],
                    validation_steps=config['inets']['model_fit']['validation_steps'],
                    validation_batch_size=config['inets']['model_fit']['validation_batch_size'],
                    validation_freq=config['inets']['model_fit']['validation_freq'],
                   )
print(history.history['val_loss'])


2022-06-06 13:17:16.317897: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-06 13:17:16.722994: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-06-06 13:17:16.723061: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 32158 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:81:00.0, compute capability: 8.6


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (6999, 1000)              7011000   
                                                                 
 re_lu (ReLU)                (6999, 1000)              0         
                                                                 
 dense_1 (Dense)             (6999, 7)                 7007      
                                                                 
Total params: 7,018,007
Trainable params: 7,018,007
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000


2022-06-06 13:17:17.185838: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1658] (One-time warning): Not using XLA:CPU for cluster.

If you want XLA:CPU, do one of the following:

 - set the TF_XLA_FLAGS to include "--tf_xla_cpu_global_jit", or
 - set cpu_global_jit to true on this session's OptimizerOptions, or
 - use experimental_jit_scope, or
 - use tf.function(jit_compile=True).

To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a
proper command-line flag, not via TF_XLA_FLAGS).
2022-06-06 13:17:17.507015: I tensorflow/compiler/xla/service/service.cc:171] XLA service 0x7f72e000a1e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-06-06 13:17:17.507071: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2022-06-06 13:17:17.511458: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:237] disabling MLIR crash reproducer, set env var

205/219 [===========================>..] - ETA: 0s - loss: 1.6675 - mse: 0.5314 - root_mean_squared_error: 0.7290

2022-06-06 13:17:20.224775: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


219/219 [==============================] - 4s 9ms/step - loss: 1.6669 - mse: 0.5309 - root_mean_squared_error: 0.7286 - val_loss: 1.6779 - val_mse: 0.5433 - val_root_mean_squared_error: 0.7371
Epoch 2/1000
219/219 [==============================] - 2s 9ms/step - loss: 1.6633 - mse: 0.5300 - root_mean_squared_error: 0.7280 - val_loss: 1.6752 - val_mse: 0.5432 - val_root_mean_squared_error: 0.7370
Epoch 3/1000
219/219 [==============================] - 1s 4ms/step - loss: 1.6605 - mse: 0.5298 - root_mean_squared_error: 0.7279 - val_loss: 1.6726 - val_mse: 0.5432 - val_root_mean_squared_error: 0.7370
Epoch 4/1000
219/219 [==============================] - 1s 4ms/step - loss: 1.6577 - mse: 0.5296 - root_mean_squared_error: 0.7277 - val_loss: 1.6700 - val_mse: 0.5432 - val_root_mean_squared_error: 0.7370
Epoch 5/1000
219/219 [==============================] - 1s 4ms/step - loss: 1.6548 - mse: 0.5293 - root_mean_squared_error: 0.7276 - val_loss: 1.6674 - val_mse: 0.5433 - val_root_mean_squar

In [ ]:
save_model(model)

2022-06-06 13:50:03.724590: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
